# Tensor product discretization

As explained in [Section 6.2](dissertation_wess.pdf#section.6.2), for the exterior discretization we need to define basis functions
\begin{align*}
b_j:\Gamma&\to\mathbb R,&j&=0,\ldots,M\\
\phi_j:\mathbb R_{\geq 0}&\to\mathbb R,&j&=0,\ldots,N,
\end{align*}
which we tensorize by using functions 
$$ \mathbf x\mapsto \phi_j\left(\xi(\mathbf x)\right)b_i\left(\hat{\mathbf x}(\mathbf x)\right).$$
For the radial basis functions $\phi_j$ we choose [infinite elements](infinite_elements.ipynb). The surface basis functions $b_j$ are traces of the interior basis functions (i.e. $H^1$-conforming finite element basis functions).

To realize the tensorization, we use $N+1$ copies of the according surface space $\hat X$, i.e. we use the space 
$$\hat X^N=\left(\hat X,\ldots,\hat X\right)$$
and write a discrete function as 
\begin{align}\sum_{j=0}^N c_j\phi_j(\xi) \hat u^j(\hat x),\tag{1}\end{align}
for discrete tangential functions $\hat u^j\in \hat X$.
 Note that, since only the first radial basis function $\phi_0$ has a non-vanishing trace at $0$ we identify the first exterior basis functions with the according interior basis functions.

In [1]:
from ngsolve import *
from netgen.geom2d import unit_square

N=10

m = Mesh(unit_square.GenerateMesh(maxh=0.1))

Gamma = m.Boundaries('right')

fes_int = H1(m,order=2)
fes_surf = H1(m,order=2,definedon=Gamma)
print('int dofs: ',fes_int.ndof)
print('surf dofs: ',fes_surf.ndof)

fes = ProductSpace(fes_int,*( (N+1)*[fes_surf] ))
print('components: ',fes.components)

int dofs:  501
surf dofs:  146
components:  [<ngsolve.comp.H1 object at 0x750e2fdd60c0>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>, <ngsolve.comp.H1 object at 0x750e2fe11f80>]


## Assembling tensor product matrices

As an example we consider the bilinear form
$$(u,v)\mapsto\int_{\mathbb R_{\geq 0}}\int_\Gamma u(\Psi(\xi,\hat x))v(\Psi(\xi,\hat x))d\xi d\hat x.$$
Using the decomposition from $(1)$ we end up with having to implement the bilinear-forms
$$\left(\phi_i \otimes \hat u^j,\phi_k\otimes \hat u^l\right)\mapsto\underbrace{\int_{\mathbb R_{\geq 0}}\phi_i(\xi)\phi_k(\xi)d\xi}_{:=\mathbf M_{k,i}}\int_\Gamma \hat u^j(\hat x)\hat u^l(\hat x)d\hat x.$$

Since the radial matrix $\mathbf M$ can be obtained as described [here](infinite_elements.ipynb), we need to assemble the bilinear-form on the space $\hat X^N$

$$\left((\hat u^0,\ldots,\hat u^N),(\hat v^0,\ldots,\hat v^N)\right)\mapsto\sum_{i,j=0}^N\mathbf M_{i,j}\int_\Gamma \hat u^j(\hat x) \hat v^i(\hat x)d\hat x$$

In NgSolve this looks as follows.

In [2]:
from infinite_elements import ie_matrices
N=5
M_ie = ie_matrices(N)[0]

u,v = fes.TnT()

M = BilinearForm(
        sum(M_ie[i,j]*u[j]*v[i]*ds(definedon=Gamma) 
            for i in range(N) for j in range(N) if abs(M_ie[i,j])>0)
        ).Assemble()
print(M.mat)

Row 0:   0: 0   4: 0   39: 0   136: 0   137: 0   147: 0   501: 0   647: 0   793: 0   939: 0   1085: 0   1231: 0   1377: 0   1523: 0   1669: 0   1815: 0   1961: 0
Row 1:   1: 0.0166667   12: 0   13: 0.00833333   48: 0   138: 0   139: -0.00208333   140: 0   171: 0   173: 0   502: -0.0166667   514: -0.00833333   637: 0.00208333   648: 0   660: 0   783: 0   794: 0   806: 0   929: 0   940: 0   952: 0   1075: 0   1086: 0   1098: 0   1221: 0   1232: 0   1244: 0   1367: 0   1378: 0   1390: 0   1513: 0   1524: 0   1536: 0   1659: 0   1670: 0   1682: 0   1805: 0   1816: 0   1828: 0   1951: 0   1962: 0   1974: 0   2097: 0
Row 2:   2: 0.0166667   21: 0.00833333   22: 0   141: -0.00208333   142: 0   196: 0   503: -0.0166667   522: -0.00833333   638: 0.00208333   649: 0   668: 0   784: 0   795: 0   814: 0   930: 0   941: 0   960: 0   1076: 0   1087: 0   1106: 0   1222: 0   1233: 0   1252: 0   1368: 0   1379: 0   1398: 0   1514: 0   1525: 0   1544: 0   1660: 0   1671: 0   1690: 0   1806: 0   1817: 0 

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


The implementation of different bilinear-forms works accordingly.